In [1]:
import linkpred
from linkpred.evaluation import Pair
import networkx as nx
import random
import itertools
from operator import itemgetter
import pickle
import matplotlib.pyplot as plt
from sklearn.metrics import auc

In [2]:
import networkx as nx
import pandas as pd
import numpy as np
import json
import cdlib
from cdlib import evaluation
from cdlib import algorithms
from cdlib import viz
from cdlib import ensemble
from cdlib import readwrite
import pickle
import pquality
import matplotlib.pyplot as plt
%matplotlib inline
from nf1 import NF1
import networkx as nx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
from operator import itemgetter
import powerlaw
from networkx.drawing.nx_agraph import graphviz_layout
import pickle
from statistics import mean, stdev
import seaborn as sns
import json
import random

Note: to be able to use all crisp methods, you need to install some additional packages:  {'graph_tool', 'wurlitzer'}


Note: to be able to use all bipartite methods, you need to install some additional packages:  {'wurlitzer'}


In [3]:
path = "../DataSet FFF/"

In [4]:
path_graph = "../DataSet FFF/Graph_data/"

In [5]:
saved_path = "../DataSet FFF/LinkPrediction_task/communities/"

In [6]:
G = nx.read_graphml(path_graph+"Real_Network.graphml")

In [7]:
nx.info(G)

'Name: \nType: Graph\nNumber of nodes: 10361\nNumber of edges: 63960\nAverage degree:  12.3463'

In [8]:
community_path = "../DataSet FFF/Community_task/"

In [9]:
with open(community_path+'louvain.pickle', 'rb') as f:
    louvain_coms = pickle.load(f)

In [10]:
for i in louvain_coms.communities[:9]:
    print(len(i))

4815
1069
954
676
462
416
206
174
112


In [11]:
def prepare_data(G, idx):
    training_edge_len = len(G.edges()) - int(len(G.edges())/100 * 20)
    test_edge_len = len(G.edges()) - training_edge_len
    
    training_edges = random.sample(G.edges, training_edge_len)
    
    training = G.edge_subgraph(training_edges)
    
    test = G.copy()
    test.remove_edges_from(training.edges())
    
    nodes = list(G.nodes())
    nodes.extend(list(test.nodes()))
    
    test = [Pair(i) for i in test.edges()]
    universe = set([Pair(i) for i in itertools.product(nodes, nodes) if i[0]!=i[1]])
    print(f" -- Storing for community {idx}")
    with open(saved_path+f'universe_{idx}.pickle', 'wb') as f:
        pickle.dump(universe, f)
    with open(saved_path+f'test_{idx}.pickle', 'wb') as f:
        pickle.dump(test, f) 
    nx.write_graphml(training, saved_path+f"training_{idx}.graphml")

In [12]:
for idx, com in enumerate(louvain_coms.communities[:9]):
    sampled_nodes = random.sample(G.subgraph(com).nodes, 500)
    prepare_data(G.subgraph(sampled_nodes), idx)

 -- Storing for community 0
 -- Storing for community 1
 -- Storing for community 2
 -- Storing for community 3


ValueError: Sample larger than population or is negative

In [13]:
def predict(idx):
    with open(saved_path+f'universe_{idx}.pickle', 'rb') as f:
        universe = pickle.load(f)
    with open(saved_path+f'test_{idx}.pickle', 'rb') as f:
        test = pickle.load(f)
    training = nx.read_graphml(saved_path+f"training_{idx}.graphml")
    
    print(f"-- For {idx} predict CommonNeighbours")
    
    cn = linkpred.predictors.CommonNeighbours(training, excluded=training.edges())
    cn_results = cn.predict()
    
    res = []
    for i, j in zip(cn_results, cn_results.values()):
        res.append([Pair(i), j])
   
    with open(saved_path+f'cn_results_{idx}.pickle', 'wb') as f:
        pickle.dump(res, f)
        
        
    print(f"-- For {idx} predict AdamicAdar")
    aa = linkpred.predictors.AdamicAdar(training, excluded=training.edges())
    aa_results = aa.predict()
    
    
    res = []
    for i, j in zip(aa_results, aa_results.values()):
        res.append([Pair(i), j])
   
    with open(saved_path+f'aa_results_{idx}.pickle', 'wb') as f:
        pickle.dump(res, f)
        
    print(f"-- For {idx} predict Jaccard")
    
    jc = linkpred.predictors.Jaccard(training, excluded=training.edges())
    jc_results = jc.predict()
    
    
    res = []
    for i, j in zip(jc_results, jc_results.values()):
        res.append([Pair(i), j])

    with open(saved_path+f'jc_results_{idx}.pickle', 'wb') as f:
        pickle.dump(res, f)
    
    print(f"-- For {idx} evaluate CommonNeighbour")
        
    cn_evaluation = linkpred.evaluation.EvaluationSheet(cn_results, test, universe)

    with open(saved_path+f'cn_evaluation_{idx}.pickle', 'wb') as f:
        pickle.dump(cn_evaluation, f) 

    print(f"-- For {idx} evaluate AdamicAdar")
        
    aa_evaluation = linkpred.evaluation.EvaluationSheet(aa_results, test, universe)

    with open(saved_path+f'aa_evaluation_{idx}.pickle', 'wb') as f:
        pickle.dump(aa_evaluation, f) 
        
        
    print(f"-- For {idx} evaluate Jaccard")
        
    jc_evaluation = linkpred.evaluation.EvaluationSheet(jc_results, test, universe)

    with open(saved_path+f'jc_evaluation_{idx}.pickle', 'wb') as f:
        pickle.dump(jc_evaluation, f) 

In [14]:
def evaluate(idx):
    
    with open(saved_path+f'cn_evaluation_{idx}.pickle', 'rb') as f:
        cn_evaluation = pickle.load(f)
    with open(saved_path+f'aa_evaluation_{idx}.pickle', 'rb') as f:
        aa_evaluation = pickle.load(f)
    with open(saved_path+f'jc_evaluation_{idx}.pickle', 'rb') as f:
        jc_evaluation = pickle.load(f)

    
    print("Area Under Roc Curve (AUROC)")
    print(f"CommonNeighbour: \t {auc(cn_evaluation.fallout(), cn_evaluation.recall())}")
    print(f"AdamicAdar: \t {auc(aa_evaluation.fallout(), aa_evaluation.recall())}")
    print(f"Jaccard: \t {auc(jc_evaluation.fallout(), jc_evaluation.recall())}")
    
    
    #plt.plot(jc_evaluation.fallout(), jc_evaluation.recall(), label="Jaccard")

    #plt.xlabel("FPR")
    #plt.ylabel("TPR")
    #plt.legend()
    #plt.show()
    

In [15]:
predict(0)

-- For 0 predict CommonNeighbours
-- For 0 predict AdamicAdar
-- For 0 predict Jaccard
-- For 0 evaluate CommonNeighbour
-- For 0 evaluate AdamicAdar
-- For 0 evaluate Jaccard


In [16]:
evaluate(0)

Area Under Roc Curve (AUROC)
CommonNeighbour: 	 0.00869204328972047
AdamicAdar: 	 0.012950649163713426
Jaccard: 	 0.005185549051248945


In [17]:
predict(1)

-- For 1 predict CommonNeighbours
-- For 1 predict AdamicAdar
-- For 1 predict Jaccard
-- For 1 evaluate CommonNeighbour
-- For 1 evaluate AdamicAdar
-- For 1 evaluate Jaccard


In [18]:
evaluate(1)

Area Under Roc Curve (AUROC)
CommonNeighbour: 	 0.006804183158254424
AdamicAdar: 	 0.010881566588580284
Jaccard: 	 0.004583483508320041


In [19]:
predict(2)

-- For 2 predict CommonNeighbours
-- For 2 predict AdamicAdar
-- For 2 predict Jaccard
-- For 2 evaluate CommonNeighbour
-- For 2 evaluate AdamicAdar
-- For 2 evaluate Jaccard


In [20]:
evaluate(2)

Area Under Roc Curve (AUROC)
CommonNeighbour: 	 0.19142282823583057
AdamicAdar: 	 0.20034573594660243
Jaccard: 	 0.1453431575001799


In [ ]:
predict(3)

In [ ]:
evaluate(3)

In [ ]:
predict(4)

In [ ]:
evaluate(4)

In [ ]:
predict(5)

In [ ]:
evaluate(5)

In [ ]:
predict(6)

In [ ]:
evaluate(6)

In [ ]:
predict(7)

In [ ]:
evaluate(7)

In [ ]:
predict(8)

In [ ]:
evaluate(8)

In [21]:
def predict2(idx):
    with open(saved_path+f'universe_{idx}.pickle', 'rb') as f:
        universe = pickle.load(f)
    with open(saved_path+f'test_{idx}.pickle', 'rb') as f:
        test = pickle.load(f)
    training = nx.read_graphml(saved_path+f"training_{idx}.graphml")
    
    print(f"-- For {idx} predict Katz")
    
    kz = linkpred.predictors.Katz(training, excluded=training.edges())
    kz_results = kz.predict()
    
    res = []
    for i, j in zip(kz_results, kz_results.values()):
        res.append([Pair(i), j])
   
    with open(saved_path+f'kz_results_{idx}.pickle', 'wb') as f:
        pickle.dump(res, f)
        
        
    print(f"-- For {idx} predict SimRank")
    simrank = linkpred.predictors.SimRank(training, excluded=training.edges())
    simrank_results = simrank.predict()
    
    
    res = []
    for i, j in zip(simrank_results, simrank_results.values()):
        res.append([Pair(i), j])
   
    with open(saved_path+f'simrank_results_{idx}.pickle', 'wb') as f:
        pickle.dump(res, f)
        

    print(f"-- For {idx} evaluate Katz")
        
    kz_evaluation = linkpred.evaluation.EvaluationSheet(kz_results, test, universe)

    with open(saved_path+f'kz_evaluation_{idx}.pickle', 'wb') as f:
        pickle.dump(kz_evaluation, f) 

    print(f"-- For {idx} evaluate SimRank")
        
    simrank_evaluation = linkpred.evaluation.EvaluationSheet(simrank_results, test, universe)

    with open(saved_path+f'simrank_evaluation_{idx}.pickle', 'wb') as f:
        pickle.dump(simrank_evaluation, f) 

In [22]:
def evaluate2(idx):
    
    with open(saved_path+f'kz_evaluation_{idx}.pickle', 'rb') as f:
        kz_evaluation = pickle.load(f)
    with open(saved_path+f'simrank_evaluation_{idx}.pickle', 'rb') as f:
        sm_evaluation = pickle.load(f)

    
    print("Area Under Roc Curve (AUROC)")
    print(f"Katz: \t {auc(kz_evaluation.fallout(), kz_evaluation.recall())}")
    print(f"SimRank: \t {auc(sm_evaluation.fallout(), sm_evaluation.recall())}")

In [23]:
predict2(0)

-- For 0 predict Katz
Computing matrix powers: [############################################################] 5/5
-- For 0 predict SimRank
-- For 0 evaluate Katz
-- For 0 evaluate SimRank


In [24]:
evaluate2(0)

Area Under Roc Curve (AUROC)
Katz: 	 0.053230839200071454
SimRank: 	 0.047963998359423304


In [25]:
predict2(1)
evaluate2(1)

-- For 1 predict Katz
Computing matrix powers: [############################################################] 5/5
-- For 1 predict SimRank
-- For 1 evaluate Katz
-- For 1 evaluate SimRank
Area Under Roc Curve (AUROC)
Katz: 	 0.09997657131903473
SimRank: 	 0.10028655646427366


In [26]:
predict2(2)
evaluate2(2)

-- For 2 predict Katz
Computing matrix powers: [############################################################] 5/5
-- For 2 predict SimRank
-- For 2 evaluate Katz
-- For 2 evaluate SimRank


KeyboardInterrupt: 

In [ ]:
predict2(3)
evaluate2(3)

In [ ]:
predict2(4)
evaluate2(4)

In [ ]:
predict2(5)
evaluate2(5)

In [ ]:
predict2(6)
evaluate2(6)

In [ ]:
predict2(7)
evaluate2(7)

In [ ]:
predict2(8)
evaluate2(8)